In [ ]:
from dataset import Dataset

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from typing import Union

In [ ]:
# Conjunto de dados Iris ================================================================================================================
wine_dataset = Dataset.from_file( 
    filepath = r"datasets\wine.data", 
    label_column = 0, 
    column_names = ["class", "Alcohol", "Malicacid", "Ash", "Alcalinity of ash", "Magnesium", "Total phenols", "Flavanoids", "Nonflavanoid phenols","Proanthocyanins", "Color intensity", "Hue", "0D280 0D315 of diluted wines", "Proline"]
).normalize().move_label_to_end()

wine_dataset

In [ ]:
wine_dataset.data

In [ ]:
wine_dataset.determination_matrix()

In [ ]:
# Separa o conjunto de dados em treinamento e teste
train_dataset, test_dataset = wine_dataset.split()

In [ ]:
def dist_euclidiana( P1 : Union[ np.ndarray, list[float] ], P2 : Union[ np.ndarray, list[float] ] ) -> float:
    distance = np.subtract( P1, P2 ) ** 2
    distance = np.sqrt( np.sum( distance ) )
    
    return distance

In [ ]:
def KNN( train_dataset : Dataset, P : np.ndarray, k : int = 3 ) -> float:
    distances = []
    nearest_class = None

    # Percorre as instâncias de treinamento
    for index, *features, classe in train_dataset:
        # Armazena o indice do elemento de treinamento e a distância dele até o ponto classificado
        distances.append( (index, dist_euclidiana( features, P )) )
    
    # Ordena a lista em função da distância
    distances = sorted( distances, key = lambda el: el[1] )

    # Obtém a posição dos k elementos mais próximos
    positions = [el[0] for i, el in enumerate(distances) if i < k]

    # Obtém a classe dos k elementos mais próximos
    classes = train_dataset.y[ positions ].to_numpy()

    # Obtém a classe com a maior moda
    classes, moda = np.unique( classes, return_counts = True )
    nearest_class = classes[ np.argmax(moda) ]
    
    return nearest_class

In [ ]:
def DMC( train_dataset : Dataset, P : np.ndarray ) -> float:
    nearest_class = None
    min_distance = float('inf')
    
    # Percorre os centroides
    for _, *features, classe in train_dataset.centroids:
        current_distance = dist_euclidiana( features, P )

        # Se a distância calculada for menor que menor distância atual, atualiza
        if current_distance < min_distance:
            min_distance = current_distance
            nearest_class = classe
    
    return nearest_class

In [ ]:
# Dimensões do dataset
m, n, k = train_dataset.shape

# Obtém um exemplo
X = train_dataset.X.iloc[0]
y = train_dataset.y.iloc[0]

# Número de neurônios? Igual ao número de classes
n_neurons = k

# Adiciona o "-1" do viés à lista de features informada
features = np.r_[X, -1]

# Inicializa a matriz de pesos para os neurônios
W = np.random.normal( size = (n_neurons, n+1) )

# Percorre os neurônios
for current_neuron in range( n_neurons ):
    # Calcula a ativação
    activation = np.dot( W[current_neuron], features )

    print(f"Neuron {current_neuron} possui a ativação {activation}")

    # Calcula a saída pelo uso da função signal
    signal = lambda activation: 1 if activation >= 0 else -1

    print(f"Neuron output: {signal(activation)}")

In [ ]:
for index, *point_test, classe in test_dataset:
    classe_prevista = DMC( train_dataset, point_test )
    print(f"{index}] Previu {classe_prevista} e era {classe} [{classe_prevista == classe}]")

train_dataset._centroids